<a href="https://colab.research.google.com/github/Umaporn19/data_viz_2022/blob/main/tsdf_pca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install required modules
!pip install --quiet -U gdown
!pip install --quiet tensorflow-graphics

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.3 MB 24.6 MB/s 
     |████████████████████████████████| 660 kB 56.5 MB/s 
     |████████████████████████████████| 1.1 MB 48.9 MB/s 
     |████████████████████████████████| 281 kB 56.8 MB/s 


In [2]:
#@title Useful imports

import numpy as np
import gdown
import h5py
import matplotlib.pyplot as plt
from skimage import measure
from tensorflow_graphics.notebooks import threejs_visualization

In [3]:
#@title Download models
url = 'https://drive.google.com/uc?id=1Im74Wbj807iOLlWQpkTHIlislvCHsSVZ'
output = 'tsdf_pca_cars_bicycles.tar.gz'
gdown.cached_download(url, output, quiet=False, postprocess=gdown.extractall)

Cached Downloading: tsdf_pca_cars_bicycles.tar.gz
Downloading...
From: https://drive.google.com/uc?id=1Im74Wbj807iOLlWQpkTHIlislvCHsSVZ
To: /root/.cache/gdown/tmpc2n2kuzb/dl
100%|██████████| 109M/109M [00:03<00:00, 35.9MB/s]


'tsdf_pca_cars_bicycles.tar.gz'

In [4]:
#@title Helper functions

class TSDFPCA(object):
  def __init__(self, model_filepath):
    with h5py.File(model_filepath, 'r') as h5_file:
      self.basis = h5_file['tsdf_basis'].value
      self.mean = h5_file['tsdf_mean'].value
      self.thickness = np.squeeze(h5_file['thicknesses'].value)
      self.offset = - np.array(self.mean.shape) * self.thickness / 2.0

  def decode_to_tsdf(self, shape_code):
    return np.tensordot(self.basis, shape_code, axes=1) + self.mean

  def decode_to_mesh(self, shape_code):
    verts, faces, normals, values = measure.marching_cubes_lewiner(self.decode_to_tsdf(shape_code),level= 0, spacing=self.thickness)
    return {"vertices": verts + self.offset, "faces": faces}

  def code_size(self):
    return self.basis.shape[-1]

def visualize_mesh(mesh):
  camera = threejs_visualization.build_perspective_camera(
        field_of_view=30, position=(0.0, -1.5, 1.0))
  camera.lookAt ( 0.0, 0.0, 0.0)
  threejs_visualization.triangular_mesh_renderer([mesh], camera=camera, width=300, height=300)

In [5]:
#@title Load and create a TSDFPCA model.
model = TSDFPCA('car_dpca20_128_ds1.h5')

AttributeError: ignored

In [ ]:
#@title Decode random code vectors and display decoded mesh.
code_size = model.code_size()
random_code = lambda : np.random.uniform(low=-40, high=40, size=code_size)

visualize_mesh(model.decode_to_mesh(np.zeros(code_size)))
visualize_mesh(model.decode_to_mesh(random_code()))
visualize_mesh(model.decode_to_mesh(random_code()))
visualize_mesh(model.decode_to_mesh(random_code()))
visualize_mesh(model.decode_to_mesh(random_code()))